In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

    

cuda:2


In [2]:
rho = 4.43
q = 200

In [3]:
loss_thresh = 25000
level = "_high"
label = "Inverse_HS" + level

In [4]:
data = scipy.io.loadmat('../inverse_HS_test5_v1.mat') 

pi = torch.from_numpy(np.array(np.pi)).double().to(device)

x = np.array(data['x'])
t = np.array(data['t'])

X,T = np.meshgrid(x,t)

X = X.flatten('F').reshape(-1,1)
T = T.flatten('F').reshape(-1,1)
  
xt = np.hstack((X,T))

u_true = np.array(data['sol'][:]).reshape(-1,1)/300.0

xt_test_tensor = torch.from_numpy(xt).float().to(device)
u_true_norm = np.linalg.norm(u_true,2)

lb_xt = xt[0]
ub_xt = xt[-1]

In [5]:
def trainingdata(N_I,N_N,N_f,seed):
    '''Boundary Conditions''' 
    
    np.random.seed(seed)
    
    x_l = x[0]*np.ones((N_N,1))
    t_l = np.random.uniform(t[0],t[-1],(N_N,1))
    xt_l = np.hstack((x_l,t_l))
    
    x_r = x[-1]*np.ones((N_N,1))
    t_r = np.random.uniform(t[0],t[-1],(N_N,1))
    xt_r = np.hstack((x_r,t_r))

    
    x_0 = np.random.uniform(x[0],x[-1],(N_I,1))
    t_0 = t[0]*np.ones((N_I,1))
    xt_0 = np.hstack((x_0,t_0))
    u_0 = 300.0*np.ones((N_I,1))/300.0
    
    xt_data = np.vstack((xt,xt_0))
    u_data = np.vstack((u_true.reshape(-1,1),u_0))
    
#     xt_data = xt
#     u_data = u_true.reshape(-1,1)
    
   
    '''Collocation Points'''

    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    samples = sampling(N_f)
    
    xt_coll = lb_xt + (ub_xt - lb_xt)*samples
    
    xt_coll = np.vstack((xt_coll, xt_r,xt_l,xt_0)) # append training points to collocation points 

    return xt_coll, xt_l,xt_r, xt_data,u_data

In [11]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()

     
        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        self.iter = 0
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)   
        
        
        self.cp = Parameter(torch.ones(1,1))
        self.cp.requiresGrad = True
        
        self.k = Parameter(torch.ones(1,1))
        self.k.requiresGrad = True
            
    'foward pass'
    def forward(self,xy):
        if torch.is_tensor(xy) != True:         
            xy = torch.from_numpy(xy)                
        
        ubxy = torch.from_numpy(ub_xt).float().to(device)
        lbxy = torch.from_numpy(lb_xt).float().to(device)
    
                      
        #preprocessing input 
        xy = 2.0*(xy - lbxy)/(ubxy - lbxy) - 1.0
        
        #convert to float
        a = xy.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a =self.activation(z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_IC(self,xt,u):
                
        loss_ic = self.loss_function(self.forward(xt), u)
                
        return loss_ic
    
    def loss_NBC_L(self,xt_NBC,N_hat):
        
        g = xt_NBC.clone()             
        g.requires_grad = True
        u = self.forward(g) 
        
        u_x_t = autograd.grad(u,g,torch.ones([xt_NBC.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]       
        
        du_dx = u_x_t[:,[0]]
        
        f = torch.abs(self.k)*du_dx + q/300.0
        
        loss_NBC = self.loss_function(f,N_hat)
                
        return loss_NBC
    
    def loss_NBC_R(self,xt_NBC,N_hat):
        
        g = xt_NBC.clone()             
        g.requires_grad = True
        u = self.forward(g) 
        
        u_x_t = autograd.grad(u,g,torch.ones([xt_NBC.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]       
        
        du_dx = u_x_t[:,[0]]
        
        f = torch.abs(self.k)*du_dx
        
        loss_NBC = self.loss_function(f,N_hat)
                
        return loss_NBC
    
    def loss_PDE(self, xt_coll, f_hat):
        
        g = xt_coll.clone()             
        g.requires_grad = True
        u = self.forward(g) 
        
        u_x_t = autograd.grad(u,g,torch.ones([xt_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        u_xx_tt = autograd.grad(u_x_t,g,torch.ones(xt_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        du_dt = u_x_t[:,[1]]
        
        d2u_dx2 = u_xx_tt[:,[0]]
        

        f = rho*torch.abs(self.cp)*du_dt - torch.abs(self.k)*d2u_dx2
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xt_IC,u_IC,xt_NBC_L,xt_NBC_R,N_hat,xt_coll,f_hat):

        loss_IC = self.loss_IC(xt_IC,u_IC)
        loss_NBC_L = self.loss_NBC_L(xt_NBC_L,N_hat)
        loss_NBC_R = self.loss_NBC_R(xt_NBC_R,N_hat)
        loss_f = self.loss_PDE(xt_coll,f_hat)
        
        loss_val = loss_IC + loss_NBC_L + loss_NBC_R + loss_f
        
        return loss_val
         
    'test neural network'
    def test(self):
        u_pred = self.forward(xt_test_tensor)
        u_pred = u_pred.cpu().detach().numpy()
   
        return u_pred

    def test_loss(self):
        u_pred = self.test()
        
        test_mse = np.mean(np.square(u_pred.reshape(-1,1) - u_true.reshape(-1,1)))
        test_re = np.linalg.norm(u_pred.reshape(-1,1) - u_true.reshape(-1,1),2)/u_true_norm
        
        
        
        return test_mse, test_re 

In [7]:
def data_update(loss_np):
    train_loss.append(loss_np)

    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [8]:
def train_step(xt_IC,u_IC,xt_NBC_L,xt_NBC_R,N_hat,xt_coll,f_hat,seed):

    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(xt_IC,u_IC,xt_NBC_L,xt_NBC_R,N_hat,xt_coll,f_hat)
        loss.backward()
        #print(loss.cpu().detach().numpy())
        return loss

    optimizer.step(closure)

In [13]:
def train_model(max_iter,rep): 
    print(rep) 
    torch.manual_seed(rep*9)
    start_time = time.time() 
    thresh_flag = 0

    xt_coll, xt_NBC_L,xt_NBC_R, xt_IC,u_IC = trainingdata(N_I,N_N,N_f,rep*22)
        
    xt_coll = torch.from_numpy(xt_coll).float().to(device)
    xt_NBC_L = torch.from_numpy(xt_NBC_L).float().to(device)
    xt_NBC_R = torch.from_numpy(xt_NBC_R).float().to(device)
    xt_IC = torch.from_numpy(xt_IC).float().to(device)
    u_IC = torch.from_numpy(u_IC).float().to(device)
        
    f_hat = torch.zeros(xt_coll.shape[0],1).to(device)
    N_hat = torch.zeros(xt_NBC_L.shape[0],1).to(device)
    

    for i in range(max_iter):
        train_step(xt_IC,u_IC,xt_NBC_L,xt_NBC_R,N_hat,xt_coll,f_hat,i)
        loss_np = PINN.loss(xt_IC,u_IC,xt_NBC_L,xt_NBC_R,N_hat,xt_coll,f_hat).cpu().detach().numpy()
        
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1          
                thresh_flag = 1       
        data_update(loss_np)
        
        print(i,"Train Loss",train_loss[-1],"Test RE",test_re_loss[-1],"cp:",PINN.cp.cpu().detach().numpy(),"K:",PINN.k.cpu().detach().numpy())   
        
    elapsed_time[rep] = time.time() - start_time  
    print('Training time: %.2f' % (elapsed_time[rep]))

In [14]:
max_reps = 3
max_iter = 200 #200

train_loss_full = []
test_mse_full = []
test_re_full = []

elapsed_time= np.zeros((max_reps,1))
time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))



N_I = 500 #Total number of data points for 'y'
N_N = 500
N_f = 10000 #Total number of collocation points 


for reps in range(max_reps):
    print(label)
    print(reps)
    train_loss = []
    test_mse_loss = []
    test_re_loss = []
 
    
    torch.manual_seed(reps*36)
    
    layers = np.array([2,50,50,50,1]) #9 hidden layers

    PINN = Sequentialmodel(layers)
   
    PINN.to(device)

    'Neural Network Summary'
    print(PINN)

    params = list(PINN.parameters())
    
    optimizer = torch.optim.LBFGS(PINN.parameters(), lr=1, 
                              max_iter = 20, 
                              max_eval = 30, 
                              tolerance_grad = 1e-08, 
                              tolerance_change = 1e-08, 
                              history_size = 100, 
                              line_search_fn = 'strong_wolfe')



   
    
    nan_flag = train_model(max_iter,reps)
    
    
    torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
    train_loss_full.append(train_loss)
    test_mse_full.append(test_mse_loss)
    test_re_full.append(test_re_loss)
    #elapsed_time[reps] = time.time() - start_time



    print('Training time: %.2f' % (elapsed_time[reps]))
    
mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "label": label,"Thresh Time": time_threshold,"Thresh epoch": epoch_threshold}
savemat(label+'.mat', mdic)

Inverse_HS_high
0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 0.088634826 Test RE 0.03822648340183399 cp: [[-0.03963499]] K: [[0.9696396]]
1 Train Loss 0.011071675 Test RE 0.018055768283680846 cp: [[-0.04777173]] K: [[1.5245544]]
2 Train Loss 0.0034033293 Test RE 0.00768680315734431 cp: [[-0.05069447]] K: [[2.7583866]]
3 Train Loss 0.0016303043 Test RE 0.007244953759918037 cp: [[-0.05129617]] K: [[3.0514808]]
4 Train Loss 0.0008712879 Test RE 0.004479023052000035 cp: [[-0.04980858]] K: [[3.8120432]]
5 Train Loss 0.0005245064 Test RE 0.0034350119974046 cp: [[-0.05028655]] K: [[4.411655]]
6 Train Loss 0.00041901838 Test RE 0.0035819836711329677 cp: [[-0.05023164]] K: [[4.2166977]]
7 T

In [ ]:
x = np.linspace(-2,2,500).reshape(-1,1)
y = np.linspace(0,1,500).reshape(-1,1)

X,Y = np.meshgrid(x,y)

X = X.flatten('F').reshape(-1,1)
Y = Y.flatten('F').reshape(-1,1)
  
xy = np.hstack((X,Y))

xy_test_tensor = torch.from_numpy(xy).float().to(device)

u_pred = PINN.forward(xy_test_tensor).cpu().detach().numpy()

In [ ]:
fig, ax = plt.subplots(1,1)
cmap = plt.cm.jet

img3 = ax.imshow(np.flip(u_pred.reshape(500,500),axis = 0),cmap = cmap,extent=[0,1,0,1],aspect = 0.75)


In [ ]:
a = 0
for i in range(10):
    a = a + test_loss_full[i][-1]
print(a/10)

In [ ]:
a = 0
for i in range(10):
    a = a + train_loss_full[i][-1]
print(a/10)